Tratamento Poligonos Bairros

In [1]:
import pandas as pd

In [34]:
df = pd.read_csv('BairrosKmlToCsv.csv')

In [35]:
xy = df[['Latitude','Longitude']]

In [36]:
df = pd.read_csv('BairroCSV.csv')

In [37]:
df[['X' , 'Y']] = xy

In [39]:
df = df.rename(columns={'X': 'Latitude', 'Y': 'Longitude'})

In [42]:
df =df[['id', 'Name', 'Popula____o', 'Domic__lios', 'Dom__Ocupados', 'Geocodigo_Setor', 'Munic__pios', 'Cor_Bairro', 'Latitude', 'Longitude']]

In [44]:
df.to_csv("BairrosCSVBI.csv")

In [1]:
import sys
print(sys.executable)

c:\Users\Desenvolvimento\AppData\Local\anaconda3\python.exe


Tratamento Marcadores Educação

In [28]:
import geopandas as gpd
import pandas as pd

In [38]:
gdf = gpd.read_file('../Data/Row/Educacao.kml', driver='KML')

In [30]:
gdf['Description'].head()

0    descrição: Tipo: Educação Especial<br>Alunos: ...
1    descrição: Tipo: Infantil / Fundamental<br>Alu...
2    descrição: Tipo: Infantil / Fundamental 1 e 2 ...
3    descrição: Tipo: <br>Alunos: 688<br>Endereço: ...
4    descrição: Tipo: <br>Alunos: 242<br>Endereço: ...
Name: Description, dtype: object

In [39]:
import re
import unicodedata
import numpy as np
import pandas as pd

def _normalize_key(key):
    key = key.strip().lower()
    # remove acentos para comparação consistente
    key = ''.join(ch for ch in unicodedata.normalize('NFD', key)
                  if unicodedata.category(ch) != 'Mn')
    return key

def parse_description(desc):
    """Retorna dict com chaves normalizadas (sem acento) e valores,
       converts empty strings to np.nan, extrai lat/lon se possível."""
    if pd.isna(desc):
        return {}

    s = str(desc)
    # transforma <br> em quebra de linha
    s = re.sub(r'(?i)<br\s*/?>', '\n', s)
    s = s.replace('\r', '\n')
    # split por linhas, strip e remove linhas vazias
    parts = [p.strip().replace('\t', ' ') for p in s.splitlines() if p.strip()]

    # remove duplicatas idênticas mantendo ordem (caso de blocos repetidos)
    seen = set()
    dedup = []
    for p in parts:
        if p not in seen:
            seen.add(p)
            dedup.append(p)
    parts = dedup

    data = {}
    for p in parts:
        if ':' not in p:
            continue
        key, val = p.split(':', 1)
        kn = _normalize_key(key)
        v = val.strip()
        if v == '':
            v = None
        data[kn] = v

    # Se existir 'descricao' cujo valor traz novos pares (ex: "Tipo: ..."), parse recursivamente
    if 'descricao' in data and data['descricao'] and ':' in str(data['descricao']):
        nested = parse_description(data['descricao'])
        # sobrescreve com valores mais internos quando existirem
        for nk, nv in nested.items():
            if nv is not None:
                data[nk] = nv
        data.pop('descricao', None)

    # extrai GPS -> lat, lon (assumimos formato "lat, lon")
    if 'gps' in data and isinstance(data['gps'], str):
        coords = re.findall(r'-?\d+\.\d+', data['gps'])
        if len(coords) >= 2:
            data['lat'] = float(coords[0])
            data['lon'] = float(coords[1])

    # normaliza missing values para np.nan
    for k in list(data.keys()):
        if data[k] is None or (isinstance(data[k], str) and data[k].strip() == ''):
            data[k] = np.nan

    return data

# --- aplicação ao GeoDataFrame (preserva geometry) ---
parsed = gdf['Description'].apply(parse_description)
parsed_df = pd.json_normalize(parsed)

# (opcional) renomear colunas para Capitalizado/Português legível
# parsed_df = parsed_df.rename(columns=lambda c: {
#     'tipo':'Tipo', 'alunos':'Alunos', 'endereco':'Endereço', 'bairro':'Bairro',
#     'gps':'GPS', 'lat':'Lat', 'lon':'Lon'}.get(c, c.capitalize()))

# junta com gdf mantendo geometry
gdf_limpo = pd.concat([gdf.drop(columns=['Description']).reset_index(drop=True),
                       parsed_df.reset_index(drop=True)], axis=1)


In [27]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Lê o CSV
df = pd.read_csv('../Data/Processed/Bairros/dens-demo-sjm.csv')

# Converte a coluna WKT em geometria
df['geometry'] = df['WKT'].apply(wkt.loads)

# Cria o GeoDataFrame com o CRS (sistema de referência geográfica)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# Remove colunas desnecessárias (opcional)
# gdf = gdf.drop(columns=['tessellate', 'extrude', 'visibility'])

# Exporta para GeoJSON
gdf.to_file('../Data/Processed/Bairros/dens-demo-sjm.geojson', driver='GeoJSON')


In [40]:
gdf_limpo

,Name,geometry,alunos,endereco,bairro,gps,tipo,lat,lon,descricao
0,CAEE - Centro de Atendimento Educacional Espec...,POINT Z (-43.40584 -22.78876 0),NaN,"Rua Elisário de Souza, 600",Vila Norma,"-22.78875620489328, -43.405836748291044",Educação Especial,-22.788756,-43.405837,NaN
1,CIEP 132 São João Bosco,POINT Z (-43.40061 -22.79107 0),889.0,Rua Torres Homem. s/n°,Éden,"-22.791066547301476, -43.40061190226758",Infantil / Fundamental,-22.791067,-43.400612,NaN
2,CIEP 138 Dauta Jobert Barreto,POINT Z (-43.38705 -22.78916 0),583.0,"Rua Joaquim Coelho Alamo, s/n°",Vila Tiradentes,"-22.78915950565353, -43.38705011576157",Infantil / Fundamental 1 e 2 / Integral,-22.789160,-43.387050,NaN
3,CIEP 180 Pres. João Goulart,POINT Z (-43.34499 -22.80121 0),688.0,"Rua Três s/n°, Pq. Juriti",Parque Araruama,"-22.801213551699096, -43.344990231102415",NaN,-22.801214,-43.344990,NaN
4,CIEP 378 Thereza Peixoto Gonçalves,POINT Z (-43.36262 -22.79813 0),242.0,Rua Piauí s/n°,Centro,"-22.79813107575488, -43.36261928877315",NaN,-22.798131,-43.362619,NaN
...,...,...,...,...,...,...,...,...,...,...
65,EM Valério Villas Boas Filho,POINT Z (-43.34883 -22.78415 0),851.0,"Rua Brasil Valério, Lt. 7A Qd 11",Venda Velha,"-22.78415304612559, -43.34882670596181",NaN,-22.784153,-43.348827,NaN
66,EM Vasco Afonso de Caravalho,POINT Z (-43.34334 -22.76213 0),477.0,"Rua Projetada, s/n°",Jardim Paraíso,"-22.762133707470557, -43.34334198877411",NaN,-22.762134,-43.343342,NaN
67,EM Vereador Celso Guerra,POINT Z (-43.39028 -22.77243 0),540.0,"Rua Urânio, n° 20",Parque Alian,"-22.772426021634292, -43.39010370596213",NaN,-22.772426,-43.390104,NaN
68,Jardim Infância Municipalizado PIPA,POINT Z (-43.37435 -22.77004 0),187.0,Rua Anchieta s/n°,Praça da Bandeira,"-22.770041325674978, -43.37434571945637",Creche,-22.770041,-43.374346,NaN


In [43]:
mapeamento = {
    "Tipo:": "Tipo: Não informado",
    "": "Tipo: Não informado",  # substitui vazio também
    "Infantil / Fundamental 1 e 2 / Integral": "Tipo: Infantil / Fundamental"
}

# Aplica substituições
gdf_limpo["tipo"] = gdf_limpo["tipo"].replace(mapeamento)

In [45]:
gdf_limpo

,Name,geometry,alunos,endereco,bairro,gps,tipo,lat,lon,descricao
0,CAEE - Centro de Atendimento Educacional Espec...,POINT Z (-43.40584 -22.78876 0),NaN,"Rua Elisário de Souza, 600",Vila Norma,"-22.78875620489328, -43.405836748291044",Educação Especial,-22.788756,-43.405837,NaN
1,CIEP 132 São João Bosco,POINT Z (-43.40061 -22.79107 0),889.0,Rua Torres Homem. s/n°,Éden,"-22.791066547301476, -43.40061190226758",Infantil / Fundamental,-22.791067,-43.400612,NaN
2,CIEP 138 Dauta Jobert Barreto,POINT Z (-43.38705 -22.78916 0),583.0,"Rua Joaquim Coelho Alamo, s/n°",Vila Tiradentes,"-22.78915950565353, -43.38705011576157",Infantil / Fundamental 1 e 2 / Integral,-22.789160,-43.387050,NaN
3,CIEP 180 Pres. João Goulart,POINT Z (-43.34499 -22.80121 0),688.0,"Rua Três s/n°, Pq. Juriti",Parque Araruama,"-22.801213551699096, -43.344990231102415",NaN,-22.801214,-43.344990,NaN
4,CIEP 378 Thereza Peixoto Gonçalves,POINT Z (-43.36262 -22.79813 0),242.0,Rua Piauí s/n°,Centro,"-22.79813107575488, -43.36261928877315",NaN,-22.798131,-43.362619,NaN
...,...,...,...,...,...,...,...,...,...,...
65,EM Valério Villas Boas Filho,POINT Z (-43.34883 -22.78415 0),851.0,"Rua Brasil Valério, Lt. 7A Qd 11",Venda Velha,"-22.78415304612559, -43.34882670596181",NaN,-22.784153,-43.348827,NaN
66,EM Vasco Afonso de Caravalho,POINT Z (-43.34334 -22.76213 0),477.0,"Rua Projetada, s/n°",Jardim Paraíso,"-22.762133707470557, -43.34334198877411",NaN,-22.762134,-43.343342,NaN
67,EM Vereador Celso Guerra,POINT Z (-43.39028 -22.77243 0),540.0,"Rua Urânio, n° 20",Parque Alian,"-22.772426021634292, -43.39010370596213",NaN,-22.772426,-43.390104,NaN
68,Jardim Infância Municipalizado PIPA,POINT Z (-43.37435 -22.77004 0),187.0,Rua Anchieta s/n°,Praça da Bandeira,"-22.770041325674978, -43.37434571945637",Creche,-22.770041,-43.374346,NaN


In [49]:
gdf_limpo['tipo'].isna().sum()

np.int64(52)

In [50]:
gdf_limpo.to_file('../Data/Processed/Educacao.geojson', driver='GeoJSON')

separando creches de escolas

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
gdf = gpd.read_file('../Data/Processed/Educacao.geojson', driver='GeoJSON')

c:\Users\Desenvolvimento\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [55]:
gdf.head()

,Name,alunos,endereco,bairro,gps,tipo,lat,lon,descricao,geometry
0,CAEE - Centro de Atendimento Educacional Espec...,None,"Rua Elisário de Souza, 600",Vila Norma,"-22.78875620489328, -43.405836748291044",Educação Especial,-22.788756,-43.405837,None,POINT Z (-43.40584 -22.78876 0)
1,CIEP 132 São João Bosco,889.0,Rua Torres Homem. s/n°,Éden,"-22.791066547301476, -43.40061190226758",Infantil / Fundamental,-22.791067,-43.400612,None,POINT Z (-43.40061 -22.79107 0)
2,CIEP 138 Dauta Jobert Barreto,583.0,"Rua Joaquim Coelho Alamo, s/n°",Vila Tiradentes,"-22.78915950565353, -43.38705011576157",Infantil / Fundamental 1 e 2 / Integral,-22.789160,-43.387050,None,POINT Z (-43.38705 -22.78916 0)
3,CIEP 180 Pres. João Goulart,688.0,"Rua Três s/n°, Pq. Juriti",Parque Araruama,"-22.801213551699096, -43.344990231102415",Infantil / Fundamental 1 e 2 / Integral,-22.801214,-43.344990,None,POINT Z (-43.34499 -22.80121 0)
4,CIEP 378 Thereza Peixoto Gonçalves,242.0,Rua Piauí s/n°,Centro,"-22.79813107575488, -43.36261928877315",Infantil / Fundamental 1 e 2 / Integral,-22.798131,-43.362619,None,POINT Z (-43.36262 -22.79813 0)


In [12]:
df_creche['urlimg'] = ""
df_escolas['urlimg'] = ""

c:\Users\Desenvolvimento\AppData\Local\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Desenvolvimento\AppData\Local\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [13]:
df_creche.to_file('../Data/Processed/Creches.geojson', driver='GeoJSON')
df_escolas.to_file('../Data/Processed/Escolas.geojson', driver='GeoJSON')

In [191]:
import pandas as pd

In [192]:
df = pd.read_excel("../Data/Row/DatasetMapeamentoTurmasSJM.xlsx", sheet_name="Dados")

In [35]:
df[['endereco', 'bairro', 'gps', 'lat', 'lon', 'geometry']] = ""

In [195]:
gdf

,Name,alunos,endereco,bairro,gps,tipo,lat,lon,descricao,geometry
0,CAEE - Centro de Atendimento Educacional Espec...,None,"Rua Elisário de Souza, 600",Vila Norma,"-22.78875620489328, -43.405836748291044",Educação Especial,-22.788756,-43.405837,None,POINT Z (-43.40584 -22.78876 0)
1,CIEP 132 São João Bosco,889.0,Rua Torres Homem. s/n°,Éden,"-22.791066547301476, -43.40061190226758",Infantil / Fundamental,-22.791067,-43.400612,None,POINT Z (-43.40061 -22.79107 0)
2,CIEP 138 Dauta Jobert Barreto,583.0,"Rua Joaquim Coelho Alamo, s/n°",Vila Tiradentes,"-22.78915950565353, -43.38705011576157",Infantil / Fundamental 1 e 2 / Integral,-22.789160,-43.387050,None,POINT Z (-43.38705 -22.78916 0)
3,CIEP 180 Pres. João Goulart,688.0,"Rua Três s/n°, Pq. Juriti",Parque Araruama,"-22.801213551699096, -43.344990231102415",Infantil / Fundamental 1 e 2 / Integral,-22.801214,-43.344990,None,POINT Z (-43.34499 -22.80121 0)
4,CIEP 378 Thereza Peixoto Gonçalves,242.0,Rua Piauí s/n°,Centro,"-22.79813107575488, -43.36261928877315",Infantil / Fundamental 1 e 2 / Integral,-22.798131,-43.362619,None,POINT Z (-43.36262 -22.79813 0)
...,...,...,...,...,...,...,...,...,...,...
65,EM Valério Villas Boas Filho,851.0,"Rua Brasil Valério, Lt. 7A Qd 11",Venda Velha,"-22.78415304612559, -43.34882670596181",Fundamental,-22.784153,-43.348827,None,POINT Z (-43.34883 -22.78415 0)
66,EM Vasco Afonso de Caravalho,477.0,"Rua Projetada, s/n°",Jardim Paraíso,"-22.762133707470557, -43.34334198877411",Infantil / Fundamental,-22.762134,-43.343342,None,POINT Z (-43.34334 -22.76213 0)
67,EM Vereador Celso Guerra,540.0,"Rua Urânio, n° 20",Parque Alian,"-22.772426021634292, -43.39010370596213",Fundamental 1 e 2,-22.772426,-43.390104,None,POINT Z (-43.39028 -22.77243 0)
68,Jardim Infância Municipalizado PIPA,187.0,Rua Anchieta s/n°,Praça da Bandeira,"-22.770041325674978, -43.37434571945637",Creche,-22.770041,-43.374346,None,POINT Z (-43.37435 -22.77004 0)


In [196]:
df

,Escola,Turma,Serie,Turno,Quantidade de Vagas,Alunos na turma,Vagas Disponiveis
0,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRÉ I - A,PRÉ-ESCOLA I EDUCAÇÃO INFANTIL,MANHÃ,27,13,14
1,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRÉ I - B,PRÉ-ESCOLA I EDUCAÇÃO INFANTIL,TARDE,27,22,5
2,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRE II D,PRÉ-ESCOLA II EDUCAÇÃO INFANTIL,TARDE,27,15,12
3,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRÉ II C,PRÉ-ESCOLA II EDUCAÇÃO INFANTIL,TARDE,27,17,10
4,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRÉ II A,PRÉ-ESCOLA II EDUCAÇÃO INFANTIL,MANHÃ,27,17,10
...,...,...,...,...,...,...,...
1096,ESCOLA MUNICIPAL VALÉRIO VILLAS BÔAS FILHO,21 A,2º ANO ENSINO FUNDAMENTAL,MANHÃ,35,35,0
1097,ESCOLA MUNICIPAL VALÉRIO VILLAS BÔAS FILHO,11 A,1º ANO ENSINO FUNDAMENTAL,MANHÃ,28,26,2
1098,ESCOLA MUNICIPAL VALÉRIO VILLAS BÔAS FILHO,11 C,1º ANO ENSINO FUNDAMENTAL,MANHÃ,20,19,1
1099,ESCOLA MUNICIPAL VALÉRIO VILLAS BÔAS FILHO,11 B,1º ANO ENSINO FUNDAMENTAL,MANHÃ,33,22,11


In [197]:
# ALTERAR NOME DA ESCOLA 1/7

#gdf["Name"] = gdf["Name"].replace("valor_antigo", "valor_novo")

#1
gdf["Name"] = gdf["Name"].replace("CIEP 132 São João Bosco", "CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO")
#2
gdf["Name"] = gdf["Name"].replace("CIEP 138 Dauta Jobert Barreto", "CIEP 138 MUNICIPALIZADO DAUTA JOBERT BARRETO")
#3
gdf["Name"] = gdf["Name"].replace("CIEP 180 Pres. João Goulart", "CIEP 180 MUNICIPALIZADO PRESIDENTE JOÃO GOULART")
#4
gdf["Name"] = gdf["Name"].replace("CIEP 378 Thereza Peixoto Gonçalves", "CIEP 378 THEREZA PEIXOTO GONÇALVES")
#5
gdf["Name"] = gdf["Name"].replace("Complexo Vila São João", "COMPLEXO EDUCACIONAL DA VILA SÃO JOÃO")
#6
gdf["Name"] = gdf["Name"].replace("Creche Prof. Antenor Lopes Martins", "CRECHE M. PROFESSOR ANTENOR LOPES MARTINS")
#7
gdf["Name"] = gdf["Name"].replace("Creche Vereador Benilton Ferreira Paes", "CRECHE M. VEREADOR BENILTON PAES")
#8
gdf["Name"] = gdf["Name"].replace("Creche CEDIAS", "CRECHE MUNICIPAL CEDIAS")
#9
gdf["Name"] = gdf["Name"].replace("Creche Criança Feliz", "CRECHE MUNICIPAL CRIANÇA FELIZ")
#10
gdf["Name"] = gdf["Name"].replace("Creche Emani do Amaral Peixoto", "CRECHE MUNICIPAL ERNANI DO AMARAL PEIXOTO")

In [198]:
# ALTERAR NOME DA ESCOLA 2/7

#gdf["Name"] = gdf["Name"].replace("valor_antigo", "valor_novo")

#1
gdf["Name"] = gdf["Name"].replace("Creche Favo de Mel", "CRECHE MUNICIPAL FAVO DE MEL")
#2
gdf["Name"] = gdf["Name"].replace("Creche Lindaura Amorim", "CRECHE MUNICIPAL LINDAURA AMORIM")
#3
gdf["Name"] = gdf["Name"].replace("Creche Maria Alves Lavouras", "CRECHE MUNICIPAL MARIA ALVES LAVOURAS")
#4
gdf["Name"] = gdf["Name"].replace("Creche Maria Ferreira Filgueiras", "CRECHE MUNICIPAL MARIA FERREIRA FILGUEIRAS")
#5
gdf["Name"] = gdf["Name"].replace("Creche Renascer", "CRECHE MUNICIPAL RENASCER")
#6
gdf["Name"] = gdf["Name"].replace("Creche Rosa de Oliveira Muiller", "CRECHE MUNICIPAL ROSA DE OLIVEIRA MUILER")
#7
gdf["Name"] = gdf["Name"].replace("Creche Santa Clara", "CRECHE MUNICIPAL SANTA CLARA")
#8
gdf["Name"] = gdf["Name"].replace("CRECHE VALDER VERMELHO", "CRECHE MUNICIPAL VALDER VERMELHO")
#9
gdf["Name"] = gdf["Name"].replace("CRECHE DA VILA SÃO JOÃO", "CRECHE MUNICIPAL VILA SÃO JOÃO")
#10
gdf["Name"] = gdf["Name"].replace("EM Maria da Gloria Katzenstein", "E. M. MARIA DA GLÓRIA MENEZES KATZENSTEIN")

In [199]:
# ALTERAR NOME DA ESCOLA 3/7

#gdf["Name"] = gdf["Name"].replace("valor_antigo", "valor_novo")

#1
gdf["Name"] = gdf["Name"].replace("Jardim Infância Municipalizado PIPA", "E.M. JARDIM DE INFÂNCIA PIPA")
#2
gdf["Name"] = gdf["Name"].replace("EM Adérito Gomes Gouveia", "EM ADÉRITO GOMES GOUVEIA")
#3
gdf["Name"] = gdf["Name"].replace("EM Amélia Barbosa de Moura", "EM AMÉLIA BARBOSA DE MOURA")
#4
gdf["Name"] = gdf["Name"].replace("EM Antonio Guedes", "EM ANTONIO GUEDES")
#5
gdf["Name"] = gdf["Name"].replace("EM Armando de Oliveira", "EM ARMANDO DE OLIVEIRA")
#6
gdf["Name"] = gdf["Name"].replace("EM Barão do Rio Branco", "EM BARÃO DO RIO BRANCO")
#7
gdf["Name"] = gdf["Name"].replace("EM Carlos Teixeira", "EM CARLOS TEIXEIRA")
#8
gdf["Name"] = gdf["Name"].replace("EM Casimiro de Abreu", "EM CASIMIRO DE ABREU")
#9
gdf["Name"] = gdf["Name"].replace("EM Denise Cerqueira", "EM DENISE CERQUEIRA")
#10
gdf["Name"] = gdf["Name"].replace("EM Dep. Lucas de Andrade Figueira", "EM DEP. LUCAS DE ANDRADE FIGUEIRA")

In [200]:
# ALTERAR NOME DA ESCOLA 4/7

#gdf["Name"] = gdf["Name"].replace("valor_antigo", "valor_novo")

#1
gdf["Name"] = gdf["Name"].replace("EM Dr. João Alves Martins", "EM DOUTOR JOÃO ALVES MARTINS")
#2
gdf["Name"] = gdf["Name"].replace("EM Dr. Getúlio de Moura", "EM DR GETÚLIO DE MOURA")
#3
gdf["Name"] = gdf["Name"].replace("EM Dr. Christovam Berberéia", "EM DR. CHRISTÓVAM BERBERÉIA")
#4
gdf["Name"] = gdf["Name"].replace("EM Edilberto Ribeiro de Castro", "EM EDILBERTO RIBEIRO DE CASTRO")
#5
gdf["Name"] = gdf["Name"].replace("EM Francisca Jeremias", "EM FRANCISCA JEREMIAS")
#6
gdf["Name"] = gdf["Name"].replace("EM Francisco Agostinho da Costa", "EM FRANCISCO AGOSTINHO DA COSTA")
#7
gdf["Name"] = gdf["Name"].replace("EM General Charles de Gaulle", "EM GENERAL CHARLES DE GAULLE")
#8
gdf["Name"] = gdf["Name"].replace("EM Profª. Graça Grijó", "EM GRAÇA GRIJÓ")
#9
gdf["Name"] = gdf["Name"].replace("EM Ignácio Lucas", "EM IGNACIO LUCAS")
#10
gdf["Name"] = gdf["Name"].replace("EM Iracema Campos Fernandes", "EM IRACEMA CAMPOS FERNANDES")

In [201]:
# ALTERAR NOME DA ESCOLA 5/7

#gdf["Name"] = gdf["Name"].replace("valor_antigo", "valor_novo")

#1
gdf["Name"] = gdf["Name"].replace("EM Jorge Cheuen", "EM JORGE CHEUEN")
#2
gdf["Name"] = gdf["Name"].replace("EM José Bonifácio", "EM JOSÉ BONIFÁCIO")
#3
gdf["Name"] = gdf["Name"].replace("EM José Luiz Viviani Gonçalves", "EM JOSÉ LUIZ VIVIANI GONÇALVES")
#4
gdf["Name"] = gdf["Name"].replace("EM José Marques Castanheira", "EM JOSÉ MARQUES CASTANHEIRA")
#5
gdf["Name"] = gdf["Name"].replace("EM Leonel de Moura Brizola", "EM LEONEL DE MOURA BRIZOLA")
#6
gdf["Name"] = gdf["Name"].replace("EM Manoel Antonio Sendas", "EM MANOEL ANTÔNIO SENDAS")
#7
gdf["Name"] = gdf["Name"].replace("EM Manoel Gonçalo", "EM MANOEL GONÇALO")
#8
gdf["Name"] = gdf["Name"].replace("EM Maria Rodrigues Cardoso", "EM MARIA RODRIGUES CARDOSO")
#9
gdf["Name"] = gdf["Name"].replace("EM Milton Rodrigues Pereira", "EM MILTON RODRIGUES PEREIRA")
#10
gdf["Name"] = gdf["Name"].replace("EM Octacílio Gonçalves da Silva", "EM OCTACÍLIO GONÇALVES")

In [202]:
# ALTERAR NOME DA ESCOLA 6/7

#gdf["Name"] = gdf["Name"].replace("valor_antigo", "valor_novo")

#1
gdf["Name"] = gdf["Name"].replace("EM Orlando Francisco", "EM ORLANDO FRANCISCO")
#2
gdf["Name"] = gdf["Name"].replace("EM Padre Paul Jean Guerry", "EM PADRE PAUL JEAN GUERRY")
#3
gdf["Name"] = gdf["Name"].replace("EM Presidente Kennedy", "EM PRESIDENTE KENNEDY")
#4
gdf["Name"] = gdf["Name"].replace("EM Prof. Jansen Pereira de Melo", "EM PROF JANSEM PEREIRA DE MELO")
#5
gdf["Name"] = gdf["Name"].replace("EM Prof. Milton Machado", "EM PROF MILTON MACHADO (COLEGIO FLUMINENSE EDEN)")
#6
gdf["Name"] = gdf["Name"].replace("EM Profª Almerida Leite de Abreu", "EM PROF. ALMERINDA LEITE DE ABREU")
#7
gdf["Name"] = gdf["Name"].replace("	EM Profª Mariza Azevedo Catarino", "EM PROF. MARIZA AZEVEDO CATARINO")
#8
gdf["Name"] = gdf["Name"].replace("EM Profª Lígia da Silva França", "EM PROFESSORA LIGIA DA SILVA FRANÇA")
#9
gdf["Name"] = gdf["Name"].replace("EM Henfil, Chico e Betinho (Márcia Branquinho)", "EM PROFESSORA MARCIA BRANQUINHO (HENFIL, CHICO E BETINHO).")
#10
gdf["Name"] = gdf["Name"].replace("EM Roberto Bonifácio de Queirós", "EM ROBERTO BONIFACIO DE QUEIROS")

In [203]:
# ALTERAR NOME DA ESCOLA 7/7

#gdf["Name"] = gdf["Name"].replace("valor_antigo", "valor_novo")

#1
gdf["Name"] = gdf["Name"].replace("EM Rui Barbosa", "EM RUI BARBOSA")
#2
gdf["Name"] = gdf["Name"].replace("EM Santo Antonio", "EM SANTO ANTONIO	")
#3
gdf["Name"] = gdf["Name"].replace("EM São João Batista", "EM SÃO JOÃO BATISTA")
#4
gdf["Name"] = gdf["Name"].replace("EM Unidade Integrada de 1° Grau", "EM UNIDADE INTEGRADA DE 1º GRAU")
#5
gdf["Name"] = gdf["Name"].replace("EM Vasco Afonso de Caravalho", "EM VASCO AFONSO DE CARVALHO")
#6
gdf["Name"] = gdf["Name"].replace("EM Vereador Celso Guerra", "EM VER. CELSO GUERRA")
#7
gdf["Name"] = gdf["Name"].replace("EM Vila São João", "EM VILA SÃO JOÃO")
#8
gdf["Name"] = gdf["Name"].replace("EM Prof. Virgílio Machado", "EM VIRGÍLIO MACHADO")
#9
gdf["Name"] = gdf["Name"].replace("EM Valério Villas Boas Filho", "ESCOLA MUNICIPAL VALÉRIO VILLAS BÔAS FILHO")
#10
gdf["Name"] = gdf["Name"].replace("valor_antigo", "valor_novo")

In [ ]:
#Unidade

gdf[gdf['Name'].str.contains("valério", case=False, na=False)]

In [ ]:
#Unidade

df[df['Escola'].str.contains("valério", case=False, na=False)]

In [204]:
# Garantir que os nomes batem
df["Escola"] = df["Escola"].str.strip()
gdf["Name"] = gdf["Name"].str.strip()

# Colunas que quero trazer do gdf
cols = ["Name", "endereco", "bairro", "gps", "lat", "lon", "geometry"]

# Merge 1:N (cada escola do gdf replicada nas turmas do df)
df_final = df.merge(gdf[cols], left_on="Escola", right_on="Name", how="left")

# Remove a coluna duplicada 'Name' (já temos 'Escola')
df_final = df_final.drop(columns=["Name"])


In [205]:
df_final.head(5)

,Escola,Turma,Serie,Turno,Quantidade de Vagas,Alunos na turma,Vagas Disponiveis,endereco,bairro,gps,lat,lon,geometry
0,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRÉ I - A,PRÉ-ESCOLA I EDUCAÇÃO INFANTIL,MANHÃ,27,13,14,Rua Torres Homem. s/n°,Éden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0)
1,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRÉ I - B,PRÉ-ESCOLA I EDUCAÇÃO INFANTIL,TARDE,27,22,5,Rua Torres Homem. s/n°,Éden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0)
2,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRE II D,PRÉ-ESCOLA II EDUCAÇÃO INFANTIL,TARDE,27,15,12,Rua Torres Homem. s/n°,Éden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0)
3,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRÉ II C,PRÉ-ESCOLA II EDUCAÇÃO INFANTIL,TARDE,27,17,10,Rua Torres Homem. s/n°,Éden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0)
4,CIEP 132 MUNICIPALIZADO SÃO JOÃO BOSCO,PRÉ II A,PRÉ-ESCOLA II EDUCAÇÃO INFANTIL,MANHÃ,27,17,10,Rua Torres Homem. s/n°,Éden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0)


In [206]:
# Filtra apenas os que têm "Creche"
df_creche = df_final[df_final["Escola"].str.contains("Creche", case=False, na=False)]

# Filtra todos os que são diferentes de "Creche"
df_escolas = df_final[~df_final["Escola"].str.contains("Creche", case=False, na=False)]

In [213]:
df_creche.to_csv('../Data/Processed/Educação/NewCreches.csv', sep=';')
df_escolas.to_csv('../Data/Processed/Educação/NewEscolas.csv', sep=';')

In [222]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Lê o CSV com pandas
df_creches = pd.read_csv('../Data/Processed/Educação/NewCreches.csv', sep=';')
df_escola  = pd.read_csv('../Data/Processed/Educação/NewEscolas.csv', sep=';')

df_creches['geometry'] = df_creches['geometry'].apply(wkt.loads)
gdf_creches = gpd.GeoDataFrame(df_creches, geometry='geometry', crs="EPSG:4326")

def safe_wkt_loads(x):
    try:
        return wkt.loads(x) if isinstance(x, str) else None
    except:
        return None

df_escola['geometry'] = df_escola['geometry'].apply(safe_wkt_loads)
gdf_escola = gpd.GeoDataFrame(df_escola, geometry='geometry', crs="EPSG:4326")


In [225]:
gdf_creches.to_file('../Data/Processed/Educação/NewCreches.geojson', driver='GeoJSON')
gdf_escola.to_file('../Data/Processed/Educação/NewEscolas.geojson', driver='GeoJSON')

In [16]:
gdf_creches = gpd.read_file('../Data/Processed/Educação/NewCreches.geojson', driver='GeoJSON')
gdf_escolas = gpd.read_file('../Data/Processed/Educação/NewEscolas.geojson', driver='GeoJSON')

c:\Users\Desenvolvimento\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(
c:\Users\Desenvolvimento\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [17]:
upgradeBairros = {
"Agostinho Porto" : "Agostinho Porto",
"Barão do Rio Branco" : "Jardim Meriti",
"Centro" : "Centro",
"Coelho da Rocha" : "Coelho da Rocha",
"Dick" : "Vilar dos Teles",
"Éden" : "Eden",
"Eden" : "Eden",
"Engenheiro Belford" : "Engenheiro Belford",
"Farrula" : "Coelho da Rocha",
"Fonte Carioca" : "Venda Velha",
"Gato Preto" : "Eden",
"Grande Rio" : "Eden",
"Indiara" : "Eden",
"Itapuã" : "Vila Rosali",
"Jardim Alegria" : "Jardim Metrópole",
"Jardim Botânico" : "Vilar dos Teles",
"Jardim Fátima" : "Jardim Paraíso",
"Jardim Íris" : "Vilar dos Teles",
"Jardim José Bonifácio" : "Vilar dos Teles",
"Jardim Jurema" : "Jardim Sumaré",
"Jardim Jurujuba" : "Jardim Sumaré",
"Jardim Limoeiro" : "Jardim Metrópole",
"Jardim Meriti" : "Jardim Meriti",
"Jardim Metrópole" : "Jardim Metrópole",
"Jardim Nóia" : "Vilar dos Teles",
"Jardim Nova Califórnia" : "Jardim Metrópole",
"Jardim Olavo Bilac" : "Jadim Sumaré",
"Jardim Paraíso" : "Jardim Paraíso",
"Jardim Santa Rosa" : "Vilar dos Teles",
"Jardim Santo Antônio" : "Vilar dos Teles",
"Jadim Sumaré" : "Jardim Sumaré",
"Jardim Sumaré" : "Jardim Sumaré",
"Lar Fluminense" : "Coelho da Rocha",
"Morro da Caixa D'água" : "Venda Velha",
"Morro da Malvina" : "Vilar dos Teles",
"Morro do Conceito" : "Jardim Meriti",
"Morro do Embaixador" : "Jardim Sumaré",
"Morro do Guarani" : "Jardim Sumaré",
"Morro do Pau Branco" : "Vilar dos Teles",
"Parque Alian" : "Coelho da Rocha",
"Parque Analândia" : "Parque Araruama",
"Parque Araruama" : "Parque Araruama",
"Parque Barreira" : "Venda Velha",
"Parque Barreto" : "Venda Velha",
"Parque Cruz Alta" : "Vilar dos Teles",
"Parque Elisabeth" : "Venda Velha",
"Parque José Bonifácio" : "Jardim Meriti",
"Parque Juriti" : "Parque Araruama",
"Parque Mirim" : "Parque Araruama",
"Parque Novo Rio" : "Parque Araruama",
"Parque Regina" : "Jardim Sumaré",
"Parque Santana" : "Parque Araruama",
"Parque São Judas Tadeu" : "Eden",
"Parque São Nicolau" : "São Mateus",
"Parque São Roque" : "Parque Araruama",
"Parque Siqueira" : "Tomazinho",
"Parque Teles de Menezes" : "Venda Velha",
"Parque Tietê" : "Parque Araruama",
"Parque Vitória" : "Jardim Sumaré",
"Praça da Bandeira" : "Vilar dos Teles",
"Parque Barão do Rio Branco" : "Vilar dos Teles",
"São Mateus" : "São Mateus",
"Sítio dos gansos" : "Vilar dos Teles",
"Tomazinho" : "Tomazinho",
"Trezentos" : "Jardim Sumaré",
"Trio de Ouro" : "Vilar dos Teles",
"Vale da Simpatia" : "Venda Velha",
"Venda Velha" : "Venda Velha",
"Vila Andorinha" : "Vilar dos Teles",
"Vila Andorinhas" : "Vilar dos Teles",
"Vila Colúmbia" : "Vilar dos Teles",
"Vila Dias Lopes" : "Vilar dos Teles",
"Vila dos Guedes" : "Jardim Metrópole",
"Vila Fluminense" : "Jardim Metrópole",
"Vila Humaitá" : "São Mateus",
"Vila Jurandir" : "Eden",
"Vila Laís" : "Parque Araruama",
"Vila Mercúrio" : "Vilar dos Teles",
"Vila Nadir" : "Vilar dos Teles",
"Vila Norma" : "Eden",
"Vila Nossa Senhora de Fátima" : "Jardim Metrópole",
"Vila Rosali" : "Vila Rosali",
"Vila Ruth" : "Vilar dos Teles",
"Vila Santa Rita" : "Venda Velha",
"Vila São João" : "Vilar dos Teles",
"Vila São José" : "Jadim Sumaré",
"Vila Saudade" : "Jardim Sumaré",
"Vila Tiradentes" : "Vila Tiradentes",
"Vila União" : "São Mateus",
"Vila Vênus" : "Vilar dos Teles",
"Vila Zulmira" : "Eden",
"Vilar dos Teles" : "Vilar dos Teles",
"Vilar Formoso" : "Venda Velha"
}

In [18]:
gdf_creches['bairro'] = gdf_creches['bairro'].map(upgradeBairros)

In [19]:
gdf_escolas['bairro'] = gdf_escolas['bairro'].map(upgradeBairros)

In [20]:
gdf_escolas[gdf_escolas['bairro'].isna()]

,Index,Polo,Escola,Turma,Serie,Turno,Quantidade de Vagas,Alunos na turma,Vagas Disponiveis,endereco,bairro,gps,lat,lon,geometry


In [22]:
gdf_escolas.to_file('../Data/Processed/Educação/NewEscolas.geojson', driver='GeoJSON')
gdf_creches.to_file('../Data/Processed/Educação/NewCreches.geojson', driver='GeoJSON')

In [6]:
df = pd.read_csv("Bairros Unificados.csv", sep=';')

--------------------------------- GEOJSON IN CSV -----------------------------------

In [1]:
import geopandas as gpd
import pandas as pd

In [23]:
gdfEscola = gpd.read_file('../Data/Processed/Educação/NewEscolas.geojson', driver='GeoJSON')
gdfEscola['Tipo'] = 'Escola'
gdfCreche = gpd.read_file('../Data/Processed/Educação/NewCreches.geojson', driver='GeoJSON')
gdfCreche['Tipo'] = 'Creche'
gdfNovasInstituicoes = gpd.read_file('../Data/Processed/Educação/NovasInstituicoes.geojson', driver='GeoJSON')
gdfNovasInstituicoes['Tipo'] = 'Nova Instituição'

gdf = pd.concat([gdfEscola, gdfCreche, gdfNovasInstituicoes], ignore_index=True)

c:\Users\Desenvolvimento\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(
c:\Users\Desenvolvimento\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(
c:\Users\Desenvolvimento\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [24]:
gdf['Polo'] = gdf['Polo'].astype('Int64')

In [4]:
gdf.head()

,Index,Polo,Escola,Turma,Serie,Turno,Quantidade de Vagas,Alunos na turma,Vagas Disponiveis,endereco,bairro,gps,lat,lon,geometry,Tipo
0,0,2,CIEP 132 MUN SÃO JOÃO BOSCO,PRÉ I - A,PRÉ-ESCOLA I EDUCAÇÃO INFANTIL,MANHÃ,27,13,14,Rua Torres Homem. s/n°,Eden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0),Escola
1,1,2,CIEP 132 MUN SÃO JOÃO BOSCO,PRÉ I - B,PRÉ-ESCOLA I EDUCAÇÃO INFANTIL,TARDE,27,22,5,Rua Torres Homem. s/n°,Eden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0),Escola
2,2,2,CIEP 132 MUN SÃO JOÃO BOSCO,PRE II D,PRÉ-ESCOLA II EDUCAÇÃO INFANTIL,TARDE,27,15,12,Rua Torres Homem. s/n°,Eden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0),Escola
3,3,2,CIEP 132 MUN SÃO JOÃO BOSCO,PRÉ II C,PRÉ-ESCOLA II EDUCAÇÃO INFANTIL,TARDE,27,17,10,Rua Torres Homem. s/n°,Eden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0),Escola
4,4,2,CIEP 132 MUN SÃO JOÃO BOSCO,PRÉ II A,PRÉ-ESCOLA II EDUCAÇÃO INFANTIL,MANHÃ,27,17,10,Rua Torres Homem. s/n°,Eden,"-22.791066547301476, -43.40061190226758",-22.791067,-43.400612,POINT Z (-43.40061 -22.79107 0),Escola


In [25]:
gdf.to_csv('../Data/Processed/Educação/EducacaoDatasetComplete.csv', sep=';', index=False)

In [6]:
gdf = gpd.read_file('../Data/Processed/Bairros/bairros_clean.geojson', driver='GeoJSON')

c:\Users\Desenvolvimento\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [7]:
df = pd.read_csv('../Data/Processed/Educação/EducacaoDatasetComplete.csv', sep=';')

In [15]:
df['bairro'].unique()

array(['Eden', 'Vila Tiradentes', 'Parque Araruama', 'Centro',
       'Vilar dos Teles', 'Jardim Sumaré', 'Jardim Metrópole',
       'Venda Velha', 'São Mateus', 'Coelho da Rocha', 'Jardim Meriti',
       'Agostinho Porto', 'Tomazinho', 'Vila Rosali', 'Jadim Sumaré',
       'Jardim Paraíso', 'Vila São João'], dtype=object)

In [28]:
gdf.to_csv('../Data/Processed/Bairros/bairros_clean.csv', sep=';')